In [41]:
def Main():
    #Get and validify sequence
    valid_mRNA = True
    seq = input("Please input your DNA or mRNA sequence: ")
    seq_str = str(seq)
    mRNA_seq = ""
    
    if ((seq_str.find("T") != -1) or (seq_str.find("c") != -1)
        or (seq_str.find("a") != -1) or (seq_str.find("g") != -1)
        or (seq_str.find("t") != -1)):
        for char in seq_str:
            if ((char == "T") or (char =="t")
                or (char == "u")):
                mRNA_seq += "U";
            elif (char == "g"): mRNA_seq += "G"
            elif (char == "c"): mRNA_seq += "C"
            elif (char == "a"): mRNA_seq += "A"
            else: mRNA_seq += char
    else: mRNA_seq = seq #if no typing corrections
    for char in mRNA_seq:
        if ((char != "A") and (char != "U")
            and (char != "C") and (char !="G")):
            valid_mRNA = False
            print("WRONG")
    if valid_mRNA == True:
        print("It's good")
        print(mRNA_seq)
    else: print("You suck")
                
    
    
 


    

In [44]:
Main()

Please input your DNA or mRNA sequence: ayutckgffgkfgbck,djsbcolejndf
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
WRONG
You suck
